Interfaz para cortar video

In [7]:
import cv2
import tkinter as tk
from tkinter import filedialog

# Función para abrir un archivo de video
def abrir_video():
    global video_path
    video_path = filedialog.askopenfilename()
    if video_path:
        cap = cv2.VideoCapture(video_path)
        global frame_count
        frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# Función para recortar el video
def recortar_video():
    inicio_segundos = int(inicio_entry.get())
    final_segundos = int(final_entry.get())
    
    # Calcular el número de cuadros
    inicio = int(inicio_segundos * 24)  # 24 FPS
    final = int(final_segundos * 24)  # 24 FPS

    if video_path:
        cap = cv2.VideoCapture(video_path)
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Usar 'mp4v' para formato MP4
        out = cv2.VideoWriter('video_recortado.mp4', fourcc, 24, (3840, 2160))  # 24 FPS y 4K

        cap.set(1, inicio)  # Establece el cuadro inicial
        while cap.isOpened():
            ret, frame = cap.read()
            if inicio <= frame_count and inicio < final:
                out.write(frame)
            inicio += 1

            if inicio == final:
                break

        cap.release()
        out.release()
        cv2.destroyAllWindows()

# Crear la ventana principal
root = tk.Tk()
root.title("Recortar Video")

video_path = ""
frame_count = 0

# Botón para abrir el video
abrir_button = tk.Button(root, text="Abrir Video", command=abrir_video)
abrir_button.pack()

# Cuadros de texto para especificar los segundos de inicio y final
inicio_label = tk.Label(root, text="Inicio (segundos):")
inicio_label.pack()
inicio_entry = tk.Entry(root)
inicio_entry.pack()

final_label = tk.Label(root, text="Final (segundos):")
final_label.pack()
final_entry = tk.Entry(root)
final_entry.pack()

# Botón para recortar el video
recortar_button = tk.Button(root, text="Recortar Video", command=recortar_video)
recortar_button.pack()

root.mainloop()

KeyboardInterrupt: 

Interfaz Funcional para el usuario, donde podra aplicar los filtros

In [2]:
from tkinter import *
from tkinter import filedialog
from PIL import Image
from PIL import ImageTk
import cv2
import imutils
import numpy as np

def elegir_visualizar_video():
    global cap, playing, frame_number
    if cap is not None:
        lblVideo.image = ""
        cap.release()
    video_path = filedialog.askopenfilename(filetypes = [
        ("all video format", ".mp4"),
        ("all video format", ".avi")])
    if len(video_path) > 0:
        lblInfoVideoPath.config(text=video_path)
        cap = cv2.VideoCapture(video_path)
        frame_number = 0
        playing = True
        visualizar()
    else:
        lblInfoVideoPath.config(text="Aún no se ha seleccionado un video")

def visualizar():
    global cap, playing, frame_number
    if cap is not None and playing:
        ret, frame = cap.read()
        if ret:
            frame = imutils.resize(frame, width=640)
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
            lower_white = np.array([0, 0, 200])
            upper_white = np.array([180, 30, 255])
            mask = cv2.inRange(hsv, lower_white, upper_white)

        # Encuentra los contornos en la máscara
            contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # Dibuja los contornos encontrados en el frame original
            cv2.drawContours(frame, contours, -1, (0, 0, 255), 2)

            im = Image.fromarray(frame)
            img = ImageTk.PhotoImage(image=im)
            lblVideo.configure(image=img)
            lblVideo.image = img
            lblVideo.after(10, visualizar)
        else:
            lblInfoVideoPath.config(text="Fin del video")
            playing = False

def pausar_video():
    global playing
    playing = False

def continuar_video():
    global playing
    playing = True
    visualizar()

def adelantar_video():
    global frame_number
    frame_number += 1
    visualizar()

def retroceder_video():
    global frame_number
    if frame_number > 0:
        frame_number -= 1
    visualizar()


cap = None
playing = False
frame_number = 0

root = Tk()
btnVisualizar = Button(root, text="Elegir y visualizar video", command=elegir_visualizar_video)
btnVisualizar.grid(column=0, row=0, padx=5, pady=5, columnspan=2)

btnPausar = Button(root, text="Pausar", command=pausar_video)
btnPausar.grid(column=0, row=3, padx=5, pady=5)

btnContinuar = Button(root, text="Continuar", command=continuar_video)
btnContinuar.grid(column=1, row=3, padx=5, pady=5)

btnAdelantar = Button(root, text="Adelantar", command=adelantar_video)
btnAdelantar.grid(column=0, row=4, padx=5, pady=5)

btnRetroceder = Button(root, text="Retroceder", command=retroceder_video)
btnRetroceder.grid(column=1, row=4, padx=5, pady=5)


# Agregar cuatro botones a la derecha del video
btnFiltroA = Button(root, text="Filtro A")
btnFiltroA.grid(column=2, row=2)

btnFiltroB = Button(root, text="Filtro B")
btnFiltroB.grid(column=3, row=2)

btnFiltroC = Button(root, text="Filtro C")
btnFiltroC.grid(column=2, row=3)

btnFiltroD = Button(root, text="Filtro D")
btnFiltroD.grid(column=3, row=3)

"""Faltan las funciones de los botones, pero serían algo asi:
# Programar las funciones de los botones

def filtro_a():
    # Implementar la función de filtro A

btnFiltroA.config(command=filtro_a)

def filtro_b():
    # Implementar la función de filtro B

btnFiltroB.config(command=filtro_b)

def filtro_c():
    # Implementar la función de filtro C

btnFiltroC.config(command=filtro_c)

def filtro_d():
    # Implementar la función de filtro D

btnFiltroD.config(command=filtro_d)"""


#vBoton video entrada
lblInfo1 = Label(root, text="Video de entrada:")
lblInfo1.grid(column=0, row=1)

lblInfoVideoPath = Label(root, text="Aún no se ha seleccionado un video")
lblInfoVideoPath.grid(column=1, row=1)

lblVideo = Label(root)
lblVideo.grid(column=0, row=2, columnspan=2)

root.mainloop()
_

''